In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


#Put your file path here

path = 'C:/Users/MaxBo/OneDrive/Documents/Green Flag Digital/GenZ Tariffs'


os.chdir(path)

In [2]:
#import skeleton of final dataset

import pandas as pd

genz = pd.read_csv('genz_save.csv')

In [4]:
genz.head()

,FIPS,Area Name (short),Area Name (long),Area Type,Tarriff Impact,Industry Diversity,City economic growth,AI growth cities,F 500 cities,Job listings growth,Cost of living - raw,Cost of living divided by salary,Fun factor* (multi-faceted)
0,12540,"Bakersfield, CA","Bakersfield, CA",Top 100 metro areas,0.000000,0.000000,7.733333,3.789428,0.000000,3.900944,4.110790,0.652169,1.982496
1,23420,"Fresno, CA","Fresno, CA",Top 100 metro areas,1.335117,3.315018,5.200000,4.605771,0.000000,4.711886,3.585182,1.118553,1.366058
2,44700,"Stockton, CA","Stockton-Lodi, CA",Top 100 metro areas,2.077915,1.515568,0.000000,0.000000,0.000000,6.448074,2.640436,1.639347,1.097793
3,47260,"Virginia Beach, VA-NC","Virginia Beach-Norfolk-Newport News, VA-NC",Top 100 metro areas,5.422980,6.130952,5.066667,7.459472,1.088743,2.503754,5.607387,2.623991,2.638889
4,13820,"Birmingham, AL","Birmingham-Hoover, AL",Top 100 metro areas,5.380786,6.364469,2.933333,5.303011,1.642982,4.660683,7.259140,3.471472,3.673896


# Variable: MoveBuddha GenZ Ranking

In [5]:
import pandas as pd

buddha = pd.read_csv('MoveBuddha.csv')

In [6]:
buddha

,Area Name (long),Score
0,"Madison, WI",3.174
1,"Fargo, ND-MN",3.066
2,"Columbus, OH",2.972
3,"Lincoln, NE",2.945
4,"Missoula, MT",2.930
...,...,...
378,"North Port-Sarasota-Bradenton, FL",1.308
379,"Punta Gorda, FL",1.275
380,"Kahului-Wailuku-Lahaina, HI",1.260
381,"Naples-Immokalee-Marco Island, FL",1.251


In [7]:
import pandas as pd

target_cities = pd.read_csv('target_cities.csv')

In [9]:
buddha_merge = target_cities.merge(buddha,on='Area Name (long)',how='left')

In [11]:
sum(buddha_merge['Score'].isnull())

5

In [12]:
buddha_merge.to_csv("buddha_merge.csv", index=False)

In [13]:
genz_factor = pd.read_csv('GenZ_Factor.csv')

In [14]:
buddha_merge = pd.read_csv('buddha_merge.csv')

In [17]:
factor_merge = target_cities.merge(genz_factor,on='Area Name (short)',how='left')

In [20]:
sum(factor_merge['Factor Score'].isnull())

28

In [21]:
factor_merge.to_csv("factor_merge.csv", index=False)

In [22]:
factor_merge = pd.read_csv('factor_merge.csv')

In [23]:
sum(factor_merge['Factor Score'].isnull())

11

In [25]:
factor_merge = factor_merge[['Area Name (long)','Factor Score']]



In [29]:
genz_scores = buddha_merge.merge(factor_merge,on='Area Name (long)',how='left')

In [30]:
genz_scores

,Area Name (short),Area Name (long),Population,Log_Population,Score,Factor Score
0,"Akron, OH","Akron, OH",698398,13.456544,1.808,4.502102
1,"Albany, NY","Albany-Schenectady-Troy, NY",904682,13.715339,1.832,4.884618
2,"Albuquerque, NM","Albuquerque, NM",922296,13.734621,1.709,3.517608
3,"Allentown, PA-NJ","Allentown-Bethlehem-Easton, PA-NJ",873555,13.680326,1.582,3.809009
4,"Atlanta, GA","Atlanta-Sandy Springs-Roswell, GA",6307261,15.657212,2.718,4.464333
...,...,...,...,...,...,...
95,"Washington, DC-VA-MD","Washington-Arlington-Alexandria, DC-VA-MD-WV",6304975,15.656850,2.790,4.364318
96,"Wichita, KS","Wichita, KS",652939,13.389239,1.755,NaN
97,"Winston, NC","Winston-Salem, NC",695630,13.452573,1.680,NaN
98,"Worcester, MA-CT","Worcester, MA-CT",866866,13.672640,2.480,4.093032


In [48]:
pop_min = genz_scores['Population'].min()
pop_max = genz_scores['Population'].max()
bins = np.linspace(pop_min, pop_max, 20)

In [49]:
genz_scores['pop_bin'] = pd.cut(genz_scores['Population'], bins=bins, labels=range(1, 20), include_lowest=True)

In [50]:
bin_means = genz_scores.groupby('pop_bin')['Factor Score'].mean()

In [51]:
#function to impute missing values
def impute_fun(row):
    if pd.isna(row['Factor Score']):
        return bin_means.loc[row['pop_bin']]
    else:
        return row['Factor Score']
    
genz_scores['Factor_Score'] = genz_scores.apply(impute_fun, axis=1)

In [52]:
sum(genz_scores['Factor_Score'].isnull())

0

In [53]:
genz_scores

,Area Name (short),Area Name (long),Population,Log_Population,Score,Factor Score,pop_bin,Factor_Score,Buddha_norm
0,"Akron, OH","Akron, OH",698398,13.456544,1.808,4.502102,1,4.502102,2.679528
1,"Albany, NY","Albany-Schenectady-Troy, NY",904682,13.715339,1.832,4.884618,1,4.884618,2.808146
2,"Albuquerque, NM","Albuquerque, NM",922296,13.734621,1.709,3.517608,1,3.517608,2.148982
3,"Allentown, PA-NJ","Allentown-Bethlehem-Easton, PA-NJ",873555,13.680326,1.582,3.809009,1,3.809009,1.468382
4,"Atlanta, GA","Atlanta-Sandy Springs-Roswell, GA",6307261,15.657212,2.718,4.464333,7,4.464333,7.556270
...,...,...,...,...,...,...,...,...,...
95,"Washington, DC-VA-MD","Washington-Arlington-Alexandria, DC-VA-MD-WV",6304975,15.656850,2.790,4.364318,7,4.364318,7.942122
96,"Wichita, KS","Wichita, KS",652939,13.389239,1.755,NaN,1,4.059227,2.395498
97,"Winston, NC","Winston-Salem, NC",695630,13.452573,1.680,NaN,1,4.059227,1.993569
98,"Worcester, MA-CT","Worcester, MA-CT",866866,13.672640,2.480,4.093032,1,4.093032,6.280815


In [54]:
#Blend scores together
#normalize each 0 to 10, take average, then normalzie again

genz_scores['Buddha_norm'] = (
    (genz_scores['Score'] - genz_scores['Score'].min()) /
    (genz_scores['Score'].max() - genz_scores['Score'].min())
) * 10

In [55]:
genz_scores['GenZ_Norm'] = (
    (genz_scores['Factor_Score'] - genz_scores['Factor_Score'].min()) /
    (genz_scores['Factor_Score'].max() - genz_scores['Factor_Score'].min())
) * 10

In [57]:
genz_scores['GenZ_Score'] = (genz_scores['Buddha_norm'] + genz_scores['GenZ_Norm']) /2

In [59]:
genz_scores['GenZ_Score_norm'] = (
    (genz_scores['GenZ_Score'] - genz_scores['GenZ_Score'].min()) /
    (genz_scores['GenZ_Score'].max() - genz_scores['GenZ_Score'].min())
) * 10

In [61]:
genz_scores = genz_scores[['Area Name (long)','GenZ_Score_norm']]

In [62]:
genz = genz.merge(genz_scores,on='Area Name (long)',how='left')

In [64]:
sum(genz['GenZ_Score_norm'].isnull())

0

In [65]:
genz.to_csv("genz_save.csv", index=False)

# Rent

In [68]:
rent = pd.read_csv('Rent.csv')

In [70]:
rent = rent[['Area Name (short)','Rent']]

In [73]:
rent.head()

,Area Name (short),Rent
0,United States,2023.723891
1,"New York, NY",3393.659110
2,"Los Angeles, CA",2973.941749
3,"Chicago, IL",2185.709320
4,"Dallas, TX",1785.314969


In [74]:
rent_100 = target_cities.merge(rent,on='Area Name (short)',how='left')

In [76]:
sum(rent_100['Rent'].isnull())

21

In [77]:
rent_100.to_csv("rent_100.csv", index=False)

In [78]:
rent_100 = pd.read_csv('rent_100.csv')

In [80]:
sum(rent_100['Rent'].isnull())

0

In [81]:
rent_100.head()

,Area Name (short),Area Name (long),Population,Log_Population,Rent
0,"Akron, OH","Akron, OH",698398,13.456544,1274.261574
1,"Albany, NY","Albany-Schenectady-Troy, NY",904682,13.715339,1647.776945
2,"Albuquerque, NM","Albuquerque, NM",922296,13.734621,1633.450774
3,"Allentown, PA-NJ","Allentown-Bethlehem-Easton, PA-NJ",873555,13.680326,1826.300472
4,"Atlanta, GA","Atlanta-Sandy Springs-Roswell, GA",6307261,15.657212,1918.618921


In [83]:
rent_100 = rent_100[['Area Name (long)','Rent']]

In [85]:
genz = genz.merge(rent_100,on='Area Name (long)',how='left')

In [87]:
target_cities.columns

Index(['Area Name (short)', 'Area Name (long)', 'Population',
       'Log_Population'],
      dtype='object')

In [89]:
target_pop = target_cities[['Area Name (long)','Population']]

In [91]:
genz = genz.merge(target_pop,on='Area Name (long)',how='left')

In [94]:
sum(genz['Population'].isnull())

0

In [95]:
genz.to_csv("genz_save.csv", index=False)

# Unemployment Rate

In [96]:
URate = pd.read_csv('UnemploymentRate.csv')

In [101]:
URate = URate.applymap(
    lambda x: x.replace(' Metropolitan Statistical Area', '') if isinstance(x, str) else x
)

In [102]:
URate.head()

,Area Name (long),URate
0,"Rapid City, SD",1.7
1,"Sioux Falls, SD-MN",1.8
2,"Bozeman, MT",2.1
3,"Burlington-South Burlington, VT",2.3
4,"Urban Honolulu, HI",2.3


In [104]:
urate_100 = target_cities.merge(URate,on='Area Name (long)',how='left')

In [107]:
sum(urate_100['URate'].isnull())

33

In [108]:
urate_100.to_csv("urate_100.csv", index=False)

In [109]:
urate_100 = pd.read_csv('urate_100.csv')

In [111]:
urate_100 = urate_100[['Area Name (long)','URate']]

In [113]:
sum(urate_100['URate'].isnull())

0

In [114]:
genz = genz.merge(urate_100,on='Area Name (long)',how='left')

In [115]:
genz.head()

,FIPS,Area Name (short),Area Name (long),Area Type,Tarriff Impact,Industry Diversity,City economic growth,AI growth cities,F 500 cities,Job listings growth,Cost of living - raw,Cost of living divided by salary,Fun factor* (multi-faceted),GenZ_Score_norm,Rent,Population,URate
0,12540,"Bakersfield, CA","Bakersfield, CA",Top 100 metro areas,0.000000,0.000000,7.733333,3.789428,0.000000,3.900944,4.110790,0.652169,1.982496,2.507665,1849.294013,913820,9.5
1,23420,"Fresno, CA","Fresno, CA",Top 100 metro areas,1.335117,3.315018,5.200000,4.605771,0.000000,4.711886,3.585182,1.118553,1.366058,2.892930,2003.680978,1180020,8.9
2,44700,"Stockton, CA","Stockton-Lodi, CA",Top 100 metro areas,2.077915,1.515568,0.000000,0.000000,0.000000,6.448074,2.640436,1.639347,1.097793,2.983587,2449.834551,800965,6.8
3,47260,"Virginia Beach, VA-NC","Virginia Beach-Norfolk-Newport News, VA-NC",Top 100 metro areas,5.422980,6.130952,5.066667,7.459472,1.088743,2.503754,5.607387,2.623991,2.638889,7.603586,1797.689213,1787169,3.6
4,13820,"Birmingham, AL","Birmingham-Hoover, AL",Top 100 metro areas,5.380786,6.364469,2.933333,5.303011,1.642982,4.660683,7.259140,3.471472,3.673896,4.310036,1435.691534,1184290,3.2


In [116]:
genz.to_csv("genz_save.csv", index=False)

# Upgrading our Income Variable

In [117]:
#First add raw RPP to GenZ dataset

RPP = pd.read_csv('RPP2.csv')

In [119]:
RPP = RPP[['Area Name (long)','RPP_All']]

In [121]:
genz = genz.merge(RPP,on='Area Name (long)',how='left')

In [123]:
sum(genz['RPP_All'].isnull())

0

# AI Exposure

In [171]:
#First we need to cross walk county data to MSA

crosswalk = pd.read_csv('Crosswalk.csv')

In [173]:
crosswalk = crosswalk[['County Code','MSA Title']]

In [179]:
crosswalk.head()

,County Code,MSA Title
0,1001,"Montgomery, AL MSA"
1,1003,"Daphne-Fairhope-Foley, AL MSA"
2,1005,"Eufaula, AL-GA MicroSA"
3,1007,"Birmingham, AL MSA"
4,1009,"Birmingham, AL MSA"


In [178]:
#AI EXPOSURE

exposure = pd.read_csv('AI_Exposure.csv',encoding='ISO-8859-1')

In [180]:
exposure.head()

,FIPS,Area,Area Name (short),total_exposed_jobs,total_jobs,exposure_share
0,1001,1001,"Autauga County, AL",4170.645162,13581.230780,0.307089
1,1003,1003,"Baldwin County, AL",28001.161530,91729.681120,0.305257
2,1005,1005,"Barbour County, AL",2121.951859,8147.616249,0.260438
3,1007,1007,"Bibb County, AL",1589.238836,5151.436449,0.308504
4,1009,1009,"Blount County, AL",3052.263713,10683.917390,0.285688


In [181]:
exposure=exposure[['FIPS','exposure_share']]

In [183]:
#Rename our columns

crosswalk.rename(columns={'County Code': 'FIPS'}, inplace=True)

C:\Users\MaxBo\AppData\Local\Temp/ipykernel_21016/4165215177.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crosswalk.rename(columns={'County Code': 'FIPS'}, inplace=True)


In [184]:
exposure_MSA = crosswalk.merge(exposure,on='FIPS',how='inner')

In [186]:
sum(exposure_MSA['exposure_share'].isnull())

0

In [187]:
exposure_MSA.head()

,FIPS,MSA Title,exposure_share
0,1001,"Montgomery, AL MSA",0.307089
1,1003,"Daphne-Fairhope-Foley, AL MSA",0.305257
2,1005,"Eufaula, AL-GA MicroSA",0.260438
3,1007,"Birmingham, AL MSA",0.308504
4,1009,"Birmingham, AL MSA",0.285688


In [188]:
exposure_MSA = exposure_MSA.applymap(
    lambda x: x.replace(' MSA', '') if isinstance(x, str) else x
)

In [189]:
exposure_MSA.head()

,FIPS,MSA Title,exposure_share
0,1001,"Montgomery, AL",0.307089
1,1003,"Daphne-Fairhope-Foley, AL",0.305257
2,1005,"Eufaula, AL-GA MicroSA",0.260438
3,1007,"Birmingham, AL",0.308504
4,1009,"Birmingham, AL",0.285688


In [192]:
exposure_MSA.rename(columns={'MSA Title': 'Area Name (long)'}, inplace=True)

In [193]:
exposure_MSA

,FIPS,Area Name (long),exposure_share
0,1001,"Montgomery, AL",0.307089
1,1003,"Daphne-Fairhope-Foley, AL",0.305257
2,1005,"Eufaula, AL-GA MicroSA",0.260438
3,1007,"Birmingham, AL",0.308504
4,1009,"Birmingham, AL",0.285688
...,...,...,...
1830,56029,"Cody, WY MicroSA",0.315419
1831,56033,"Sheridan, WY MicroSA",0.323323
1832,56037,"Rock Springs, WY MicroSA",0.289100
1833,56039,"Jackson, WY-ID MicroSA",0.293848


In [195]:
exposure_100 = target_cities.merge(exposure_MSA,on='Area Name (long)',how='left')

In [197]:
sum(exposure_100['exposure_share'].isnull())

34

In [198]:
exposure_100.to_csv("exposure_100.csv", index=False)

In [199]:
exposure_MSA.to_csv("exposure_MSA.csv", index=False)

In [ ]:
#A single MSA can include multiple counties. We will group by all the counties that fall within the MSA and average them.

In [231]:

target_cities_AI = pd.read_csv('target_cities_AI.csv')

In [232]:
exposure_100_2 = target_cities_AI.merge(exposure_MSA,on='Area Name (long)',how='left')

In [233]:
sum(exposure_100_2['exposure_share'].isnull())

4

In [234]:
#Connecticut exposures not included. We will impute again by population.

exposure_100_2

,Area Name (short),Area Name (long),Population,Log_Population,FIPS_x,FIPS_y,exposure_share
0,"Akron, OH","Akron, OH",698398,13.456544,10420,39133.0,0.326608
1,"Akron, OH","Akron, OH",698398,13.456544,10420,39153.0,0.340430
2,"Albany, NY","Albany-Schenectady-Troy, NY",904682,13.715339,10580,36001.0,0.383251
3,"Albany, NY","Albany-Schenectady-Troy, NY",904682,13.715339,10580,36083.0,0.340733
4,"Albany, NY","Albany-Schenectady-Troy, NY",904682,13.715339,10580,36091.0,0.342116
...,...,...,...,...,...,...,...
585,"Wichita, KS","Wichita, KS",652939,13.389239,48620,20191.0,0.310202
586,"Winston, NC","Worcester, MA",695630,13.452573,49180,25027.0,0.345613
587,"Worcester, MA-CT","Worcester, MA-CT",866866,13.672640,49340,NaN,NaN
588,"Youngstown, OH-PA","Youngstown-Warren, OH",425969,12.962122,49660,39099.0,0.307709


In [235]:
#First group by

grouped_exposure = exposure_100_2.groupby('Area Name (long)')['exposure_share'].mean()

In [236]:
grouped_exposure = grouped_exposure.to_frame(name='AI Exposure').reset_index()

In [265]:
grouped_exposure

,Area Name (long),AI Exposure
0,"Akron, OH",0.333519
1,"Albany-Schenectady-Troy, NY",0.348274
2,"Albuquerque, NM",0.329887
3,"Allentown-Bethlehem-Easton, PA-NJ",0.312858
4,"Atlanta-Sandy Springs-Roswell, GA",0.324270
...,...,...
95,"Washington-Arlington-Alexandria, DC-VA-MD-WV",0.354736
96,"Wichita, KS",0.313614
97,"Worcester, MA",0.345613
98,"Worcester, MA-CT",NaN


In [266]:
sum(grouped_exposure['AI Exposure'].isnull())

4

In [267]:
grouped_exposure_100 = target_cities_AI.merge(grouped_exposure,on='Area Name (long)',how='left')

In [268]:
grouped_exposure_100

,Area Name (short),Area Name (long),Population,Log_Population,FIPS,AI Exposure
0,"Akron, OH","Akron, OH",698398,13.456544,10420,0.333519
1,"Albany, NY","Albany-Schenectady-Troy, NY",904682,13.715339,10580,0.348274
2,"Albuquerque, NM","Albuquerque, NM",922296,13.734621,10740,0.329887
3,"Allentown, PA-NJ","Allentown-Bethlehem-Easton, PA-NJ",873555,13.680326,10900,0.312858
4,"Atlanta, GA","Atlanta-Sandy Springs-Roswell, GA",6307261,15.657212,12060,0.324270
...,...,...,...,...,...,...
95,"Washington, DC-VA-MD","Washington-Arlington-Alexandria, DC-VA-MD-WV",6304975,15.656850,47900,0.354736
96,"Wichita, KS","Wichita, KS",652939,13.389239,48620,0.313614
97,"Winston, NC","Worcester, MA",695630,13.452573,49180,0.345613
98,"Worcester, MA-CT","Worcester, MA-CT",866866,13.672640,49340,NaN


In [269]:
pop_min = grouped_exposure_100['Population'].min()
pop_max = grouped_exposure_100['Population'].max()
bins = np.linspace(pop_min, pop_max, 41)

In [270]:
grouped_exposure_100['pop_bin'] = pd.cut(grouped_exposure_100['Population'], bins=bins, labels=range(1, 41), include_lowest=True)

In [271]:
bin_means = grouped_exposure_100.groupby('pop_bin')['AI Exposure'].mean()

In [272]:
#function to impute missing values
def impute_fun(row):
    if pd.isna(row['AI Exposure']):
        return bin_means.loc[row['pop_bin']]
    else:
        return row['AI Exposure']
    
grouped_exposure_100['AI Exposure'] = grouped_exposure_100.apply(impute_fun, axis=1)

In [273]:
sum(grouped_exposure_100['AI Exposure'].isnull())

0

In [274]:
grouped_exposure_100['AI Exposure_norm'] = (
    (grouped_exposure_100['AI Exposure'] - grouped_exposure_100['AI Exposure'].min()) /
    (grouped_exposure_100['AI Exposure'].max() - grouped_exposure_100['AI Exposure'].min())
) * 10

In [275]:
grouped_exposure_100 = grouped_exposure_100[['AI Exposure','AI Exposure_norm','FIPS']]

In [276]:
grouped_exposure_100

,AI Exposure,AI Exposure_norm,FIPS
0,0.333519,5.204906,10420
1,0.348274,6.834764,10580
2,0.329887,4.803682,10740
3,0.312858,2.922637,10900
4,0.324270,4.183185,12060
...,...,...,...
95,0.354736,7.548583,47900
96,0.313614,3.006086,48620
97,0.345613,6.540823,49180
98,0.321733,3.902969,49340


In [277]:
genz = genz.merge(grouped_exposure_100,on='FIPS',how='left')

In [278]:
genz.head()

,FIPS,Area Name (short),Area Name (long),Area Type,Tarriff Impact,Industry Diversity,City economic growth,AI growth cities,F 500 cities,Job listings growth,Cost of living - raw,Cost of living divided by salary,Fun factor* (multi-faceted),GenZ_Score_norm,Rent,Population,URate,AI Exposure,AI Exposure_norm
0,12540,"Bakersfield, CA","Bakersfield, CA",Top 100 metro areas,0.000000,0.000000,7.733333,3.789428,0.000000,3.900944,4.110790,0.652169,1.982496,2.507665,1849.294013,913820,9.5,0.289240,0.313643
1,23420,"Fresno, CA","Fresno, CA",Top 100 metro areas,1.335117,3.315018,5.200000,4.605771,0.000000,4.711886,3.585182,1.118553,1.366058,2.892930,2003.680978,1180020,8.9,0.295887,1.047876
2,44700,"Stockton, CA","Stockton-Lodi, CA",Top 100 metro areas,2.077915,1.515568,0.000000,0.000000,0.000000,6.448074,2.640436,1.639347,1.097793,2.983587,2449.834551,800965,6.8,0.286401,0.000000
3,47260,"Virginia Beach, VA-NC","Virginia Beach-Norfolk-Newport News, VA-NC",Top 100 metro areas,5.422980,6.130952,5.066667,7.459472,1.088743,2.503754,5.607387,2.623991,2.638889,7.603586,1797.689213,1787169,3.6,0.323636,4.113218
4,13820,"Birmingham, AL","Birmingham-Hoover, AL",Top 100 metro areas,5.380786,6.364469,2.933333,5.303011,1.642982,4.660683,7.259140,3.471472,3.673896,4.310036,1435.691534,1184290,3.2,0.310692,2.683329


In [279]:
sum(genz['GenZ_Score_norm'].isnull())

0

In [280]:
genz['GenZ_Score_norm'].describe()

count    100.000000
mean       5.206283
std        1.988155
min        0.000000
25%        3.898001
50%        5.256644
75%        6.738477
max       10.000000
Name: GenZ_Score_norm, dtype: float64

In [282]:
genz.to_csv("genz_save3.csv", index=False)

# Upgrade Entry Level Variable

In [291]:
import pandas as pd

# Example: a single column Excel export
entry = pd.read_csv('EntryLevel.csv')


In [292]:
cities = entry.iloc[::2].reset_index(drop=True)        # Even-indexed rows: 0, 2, 4,...
percentages = entry.iloc[1::2].reset_index(drop=True) 

In [299]:
cities.rename(columns={'Column': 'Area Name (short)'}, inplace=True)

In [300]:
entry_level = pd.concat([cities, percentages], axis=1)

In [302]:
entry_level.head()

,Area Name (short),Column
0,"Akron, OH",63.00%
1,"Albany, NY",64.30%
2,"Albuquerque, NM",65.20%
3,"Allentown, PA",62.30%
4,"Anaheim, CA",49.60%


In [303]:
entry_level.rename(columns={'Column': 'Entry Level % Above'}, inplace=True)

In [304]:
entry_level.head()

,Area Name (short),Entry Level % Above
0,"Akron, OH",63.00%
1,"Albany, NY",64.30%
2,"Albuquerque, NM",65.20%
3,"Allentown, PA",62.30%
4,"Anaheim, CA",49.60%


In [306]:
entry_100 = target_cities.merge(entry_level,on='Area Name (short)',how='left')

In [308]:
sum(entry_100['Entry Level % Above'].isnull())

36

In [309]:
entry_100.to_csv("entry_100.csv", index=False)

In [343]:
entry_100 = pd.read_csv('entry_100.csv')

In [344]:
entry_100.head()

,Area Name (short),Area Name (long),Population,Log_Population,FIPS,Entry Level % Above
0,"Akron, OH","Akron, OH",698398,13.456544,10420,0.630
1,"Albany, NY","Albany-Schenectady-Troy, NY",904682,13.715339,10580,0.643
2,"Albuquerque, NM","Albuquerque, NM",922296,13.734621,10740,0.652
3,"Allentown, PA-NJ","Allentown-Bethlehem-Easton, PA-NJ",873555,13.680326,10900,0.623
4,"Atlanta, GA","Atlanta-Sandy Springs-Roswell, GA",6307261,15.657212,12060,0.712


In [345]:
#See if population is an indicator of % above average entry level income

import statsmodels.api as sm

entry_100_clean = entry_100.dropna()

# Define independent variables and dependent variable
X = entry_100_clean['Log_Population']
y = entry_100_clean['Entry Level % Above']

# Add constant for intercept
X = sm.add_constant(X)

# Fit regression model
model = sm.OLS(y, X).fit()

# Print summary
print(model.summary())

                             OLS Regression Results                            
Dep. Variable:     Entry Level % Above   R-squared:                       0.028
Model:                             OLS   Adj. R-squared:                  0.017
Method:                  Least Squares   F-statistic:                     2.491
Date:                 Mon, 19 May 2025   Prob (F-statistic):              0.118
Time:                         16:06:10   Log-Likelihood:                 83.934
No. Observations:                   88   AIC:                            -163.9
Df Residuals:                       86   BIC:                            -158.9
Df Model:                            1                                         
Covariance Type:             nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.8829      0.1

C:\Users\MaxBo\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


In [346]:
#We have a weak relationship between population and % above entry level income. Imputation based off population wouldn't 
#capture very accurate scores here. 

In [347]:
#Lets try a multionomial regression wit salary

salaries = pd.read_csv('MSA_salary2.csv')

In [348]:
salaries = salaries[['Area Name (long)','2023_salary']]

In [349]:
entry_100 = entry_100.merge(salaries,on='Area Name (long)',how='left')

In [350]:
entry_100

,Area Name (short),Area Name (long),Population,Log_Population,FIPS,Entry Level % Above,2023_salary
0,"Akron, OH","Akron, OH",698398,13.456544,10420,0.630,62904.0
1,"Albany, NY","Albany-Schenectady-Troy, NY",904682,13.715339,10580,0.643,71972.0
2,"Albuquerque, NM","Albuquerque, NM",922296,13.734621,10740,0.652,57278.0
3,"Allentown, PA-NJ","Allentown-Bethlehem-Easton, PA-NJ",873555,13.680326,10900,0.623,66493.0
4,"Atlanta, GA","Atlanta-Sandy Springs-Roswell, GA",6307261,15.657212,12060,0.712,68027.0
...,...,...,...,...,...,...,...
95,"Washington, DC-VA-MD","Washington-Arlington-Alexandria, DC-VA-MD-WV",6304975,15.656850,47900,0.264,89396.0
96,"Wichita, KS","Wichita, KS",652939,13.389239,48620,0.573,60375.0
97,"Winston, NC","Winston-Salem, NC",695630,13.452573,49180,0.732,57101.0
98,"Worcester, MA-CT","Worcester, MA-CT",866866,13.672640,49340,0.471,67810.0


In [351]:
#See if population is an indicator of % above average entry level income

import statsmodels.api as sm

entry_100_clean = entry_100.dropna()

# Define independent variables and dependent variable
X = entry_100_clean[['Log_Population','2023_salary']]
y = entry_100_clean['Entry Level % Above']

# Add constant for intercept
X = sm.add_constant(X)

# Fit regression model
model = sm.OLS(y, X).fit()

# Print summary
print(model.summary())

                             OLS Regression Results                            
Dep. Variable:     Entry Level % Above   R-squared:                       0.043
Model:                             OLS   Adj. R-squared:                  0.021
Method:                  Least Squares   F-statistic:                     1.919
Date:                 Mon, 19 May 2025   Prob (F-statistic):              0.153
Time:                         16:06:12   Log-Likelihood:                 84.620
No. Observations:                   88   AIC:                            -163.2
Df Residuals:                       85   BIC:                            -155.8
Df Model:                            2                                         
Covariance Type:             nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.9044      0.1

C:\Users\MaxBo\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


In [352]:
#average salary is also insignificant

#we attempt another relationship. The combined GenZ score from movebuddha and genz score.

genzfactornorm = pd.read_csv('GenZFactor_norm.csv')

In [353]:
entry_100 = entry_100.merge(genzfactornorm,on='Area Name (long)',how='left')

In [354]:
import statsmodels.api as sm

entry_100_clean = entry_100.dropna()

# Define independent variables and dependent variable
X = entry_100_clean['GenZ_Score_norm']
y = entry_100_clean['Entry Level % Above']

# Add constant for intercept
X = sm.add_constant(X)

# Fit regression model
model = sm.OLS(y, X).fit()

# Print summary
print(model.summary())

                             OLS Regression Results                            
Dep. Variable:     Entry Level % Above   R-squared:                       0.013
Model:                             OLS   Adj. R-squared:                  0.001
Method:                  Least Squares   F-statistic:                     1.119
Date:                 Mon, 19 May 2025   Prob (F-statistic):              0.293
Time:                         16:06:13   Log-Likelihood:                 83.247
No. Observations:                   88   AIC:                            -162.5
Df Residuals:                       86   BIC:                            -157.5
Df Model:                            1                                         
Covariance Type:             nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.6683      

C:\Users\MaxBo\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


In [355]:
#GenZ factor score was not a strong predictor.

In [356]:
#log population was our best bet. The p-value was .11 meaning if the 
#true coefficient were actually zero, there is an 11% chance you would observe a 
#coefficient at least as extreme as the one you found, just due to random noise.

In [357]:
#Also we are only imputing a small amount.


pop_min = entry_100['Log_Population'].min()
pop_max = entry_100['Log_Population'].max()
bins = np.linspace(pop_min, pop_max, 20)

In [358]:
entry_100['pop_bin'] = pd.cut(entry_100['Log_Population'], bins=bins, labels=range(1, 20), include_lowest=True)

In [359]:
bin_means = entry_100.groupby('pop_bin')['Entry Level % Above'].mean()

In [360]:
#function to impute missing values
def impute_fun(row):
    if pd.isna(row['Entry Level % Above']):
        return bin_means.loc[row['pop_bin']]
    else:
        return row['Entry Level % Above']
    
entry_100['Entry Level % Above'] = entry_100.apply(impute_fun, axis=1)

In [361]:
sum(entry_100['Entry Level % Above'].isnull())

0

In [362]:
entry_100 = entry_100[['Area Name (long)','Entry Level % Above']]

In [363]:
entry_100['Entry Level % Above_norm'] = (
    (entry_100['Entry Level % Above'] - entry_100['Entry Level % Above'].min()) /
    (entry_100['Entry Level % Above'].max() - entry_100['Entry Level % Above'].min())
) * 10

In [364]:
entry_100

,Area Name (long),Entry Level % Above,Entry Level % Above_norm
0,"Akron, OH",0.630,6.267123
1,"Albany-Schenectady-Troy, NY",0.643,6.489726
2,"Albuquerque, NM",0.652,6.643836
3,"Allentown-Bethlehem-Easton, PA-NJ",0.623,6.147260
4,"Atlanta-Sandy Springs-Roswell, GA",0.712,7.671233
...,...,...,...
95,"Washington-Arlington-Alexandria, DC-VA-MD-WV",0.264,0.000000
96,"Wichita, KS",0.573,5.291096
97,"Winston-Salem, NC",0.732,8.013699
98,"Worcester, MA-CT",0.471,3.544521


In [365]:
genz = pd.read_csv('genz_save3.csv')

In [367]:
genz = genz.merge(entry_100,on='Area Name (long)',how='left')

In [370]:
genz['Entry Level % Above_norm'].describe()

count    100.000000
mean       6.319503
std        1.556222
min        0.000000
25%        5.376712
50%        6.378425
75%        7.217466
max       10.000000
Name: Entry Level % Above_norm, dtype: float64

In [371]:
genz.to_csv("genz_save3.csv", index=False)

In [372]:
genz

,FIPS,Area Name (short),Area Name (long),Area Type,Tarriff Impact,Industry Diversity,City economic growth,AI growth cities,F 500 cities,Job listings growth,...,Cost of living divided by salary,Fun factor* (multi-faceted),GenZ_Score_norm,Rent,Population,URate,AI Exposure,AI Exposure_norm,Entry Level % Above,Entry Level % Above_norm
0,10420,"Akron, OH","Akron, OH",Top 100 metro areas,6.969572,7.156593,2.266667,3.333245,4.179065,1.690213,...,3.045155,2.172755,5.061183,1274.261574,698398,5.5,0.333519,5.204906,0.630,6.267123
1,10580,"Albany, NY","Albany-Schenectady-Troy, NY",Top 100 metro areas,8.091453,5.787546,4.133333,5.203488,0.000000,2.744517,...,3.680022,2.803991,6.129445,1647.776945,904682,3.5,0.348274,6.834764,0.643,6.489726
2,10740,"Albuquerque, NM","Albuquerque, NM",Top 100 metro areas,8.048704,6.192766,4.000000,6.085123,0.000000,3.497676,...,2.305571,2.701674,2.193224,1633.450774,922296,3.5,0.329887,4.803682,0.652,6.643836
3,10900,"Allentown, PA-NJ","Allentown-Bethlehem-Easton, PA-NJ",Top 100 metro areas,6.864109,5.254121,5.733333,3.822448,2.227412,4.504602,...,2.988044,2.803991,2.544465,1826.300472,873555,4.2,0.312858,2.922637,0.623,6.147260
4,12060,"Atlanta, GA","Atlanta-Sandy Springs-Roswell, GA",Top 100 metro areas,8.239089,8.983516,3.600000,7.867483,2.467971,6.184179,...,3.007339,6.495434,7.860564,1918.618921,6307261,3.6,0.324270,4.183185,0.712,7.671233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,47900,"Washington, DC-VA-MD","Washington-Arlington-Alexandria, DC-VA-MD-WV",Top 100 metro areas,9.510323,4.310897,4.400000,10.000000,2.931778,3.141133,...,4.614892,5.026636,7.830467,2429.474580,6304975,3.5,0.354736,7.548583,0.264,0.000000
96,48620,"Wichita, KS","Wichita, KS",Top 100 metro areas,7.852005,5.521978,4.533333,5.194047,0.000000,3.225876,...,2.986636,2.762557,3.744081,1100.123409,652939,4.0,0.313614,3.006086,0.573,5.291096
97,49180,"Winston, NC","Winston-Salem, NC",Top 100 metro areas,5.677248,5.054945,2.533333,3.712683,0.000000,4.157608,...,2.431923,2.446728,3.505292,1669.650079,695630,3.7,0.345613,6.540823,0.732,8.013699
98,49340,"Worcester, MA-CT","Worcester, MA-CT",Top 100 metro areas,6.595042,5.249542,1.466667,4.428149,2.244600,2.950240,...,2.838746,1.493531,6.140037,2174.496129,866866,5.1,0.321733,3.902969,0.471,3.544521


# Cybersecurity

In [375]:
cyber = pd.read_csv('CyberSecurity.csv')

In [376]:
cyber

,Area Name (long),Annual median wage (2),"Employment per 1,000 jobs"
0,"Akron, OH (00-10420)",101130.00,0.8070
1,Alaska nonmetropolitan area (02-00006),97610.00,0.3750
2,"Albany-Schenectady-Troy, NY (00-10580)",103590.00,0.9780
3,"Albuquerque, NM (00-10740)",136290.00,2.4250
4,"Allentown-Bethlehem-Easton, PA-NJ (00-10900)",119580.00,0.3350
...,...,...,...
296,"Winston-Salem, NC (00-49180)",123700.00,1.3280
297,"Worcester, MA (00-49340)",114530.00,0.6280
298,"York-Hanover, PA (00-49620)",84070.00,0.2880
299,"Youngstown-Warren, OH (00-49660)",70950.00,0.1920


In [377]:
cyber['Area Name (long)'] = cyber['Area Name (long)'].str.replace(r'([A-Z]{2})\s*\(.*', r'\1', regex=True)

In [378]:
cyber

,Area Name (long),Annual median wage (2),"Employment per 1,000 jobs"
0,"Akron, OH",101130.00,0.8070
1,Alaska nonmetropolitan area (02-00006),97610.00,0.3750
2,"Albany-Schenectady-Troy, NY",103590.00,0.9780
3,"Albuquerque, NM",136290.00,2.4250
4,"Allentown-Bethlehem-Easton, PA-NJ",119580.00,0.3350
...,...,...,...
296,"Winston-Salem, NC",123700.00,1.3280
297,"Worcester, MA",114530.00,0.6280
298,"York-Hanover, PA",84070.00,0.2880
299,"Youngstown-Warren, OH",70950.00,0.1920


In [380]:
cyber_100 = target_cities.merge(cyber,on='Area Name (long)',how='left')

In [383]:
sum(cyber_100['Annual median wage (2)'].isnull())

33

In [384]:
cyber_100.to_csv("cyber_100.csv", index=False)

In [417]:
cyber_100 = pd.read_csv('cyber_100.csv')

In [418]:
cyber_100.head()

,Area Name (short),Area Name (long),Population,Log_Population,FIPS,Annual median wage (2),"Employment per 1,000 jobs"
0,"Akron, OH","Akron, OH",698398,13.456544,10420,101130.0,0.807
1,"Albany, NY","Albany-Schenectady-Troy, NY",904682,13.715339,10580,103590.0,0.978
2,"Albuquerque, NM","Albuquerque, NM",922296,13.734621,10740,136290.0,2.425
3,"Allentown, PA-NJ","Allentown-Bethlehem-Easton, PA-NJ",873555,13.680326,10900,119580.0,0.335
4,"Atlanta, GA","Atlanta-Sandy Springs-Roswell, GA",6307261,15.657212,12060,126880.0,1.720


In [419]:
cyber_100_clean = cyber_100.dropna()

# Define independent variables and dependent variable
X = cyber_100_clean['Log_Population']
y = cyber_100_clean['Annual median wage (2)']

# Add constant for intercept
X = sm.add_constant(X)

# Fit regression model
model = sm.OLS(y, X).fit()

# Print summary
print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Annual median wage (2)   R-squared:                       0.129
Model:                                OLS   Adj. R-squared:                  0.119
Method:                     Least Squares   F-statistic:                     13.88
Date:                    Mon, 19 May 2025   Prob (F-statistic):           0.000332
Time:                            16:58:21   Log-Likelihood:                -1070.4
No. Observations:                      96   AIC:                             2145.
Df Residuals:                          94   BIC:                             2150.
Df Model:                               1                                         
Covariance Type:                nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
cons

C:\Users\MaxBo\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


In [420]:
#statistically significant relationship

In [421]:
cyber_100_clean = cyber_100.dropna()

# Define independent variables and dependent variable
X = cyber_100_clean['Log_Population']
y = cyber_100_clean['Employment per 1,000 jobs']

# Add constant for intercept
X = sm.add_constant(X)

# Fit regression model
model = sm.OLS(y, X).fit()

# Print summary
print(model.summary())

                                OLS Regression Results                               
Dep. Variable:     Employment per 1,000 jobs   R-squared:                       0.126
Model:                                   OLS   Adj. R-squared:                  0.117
Method:                        Least Squares   F-statistic:                     13.59
Date:                       Mon, 19 May 2025   Prob (F-statistic):           0.000381
Time:                               16:58:22   Log-Likelihood:                -96.261
No. Observations:                         96   AIC:                             196.5
Df Residuals:                             94   BIC:                             201.7
Df Model:                                  1                                         
Covariance Type:                   nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------

C:\Users\MaxBo\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


In [422]:
#Also significant. We can impute both columns based on log population.

In [423]:
pop_min = cyber_100['Log_Population'].min()
pop_max = cyber_100['Log_Population'].max()
bins = np.linspace(pop_min, pop_max, 20)

In [424]:
cyber_100['pop_bin'] = pd.cut(cyber_100['Log_Population'], bins=bins, labels=range(1, 20), include_lowest=True)

In [425]:
bin_means = cyber_100.groupby('pop_bin')['Annual median wage (2)'].mean()

In [426]:
#function to impute missing values
def impute_fun(row):
    if pd.isna(row['Annual median wage (2)']):
        return bin_means.loc[row['pop_bin']]
    else:
        return row['Annual median wage (2)']
    
cyber_100['Annual median wage (2)_impute'] = cyber_100.apply(impute_fun, axis=1)

In [427]:
sum(cyber_100['Annual median wage (2)_impute'].isnull())

0

In [428]:
pop_min = cyber_100['Log_Population'].min()
pop_max = cyber_100['Log_Population'].max()
bins = np.linspace(pop_min, pop_max, 20)

In [429]:
cyber_100['pop_bin'] = pd.cut(cyber_100['Log_Population'], bins=bins, labels=range(1, 20), include_lowest=True)

In [430]:
bin_means = cyber_100.groupby('pop_bin')['Employment per 1,000 jobs'].mean()

In [431]:
#function to impute missing values
def impute_fun(row):
    if pd.isna(row['Employment per 1,000 jobs']):
        return bin_means.loc[row['pop_bin']]
    else:
        return row['Employment per 1,000 jobs']
    
cyber_100['Employment per 1,000 jobs_impute'] = cyber_100.apply(impute_fun, axis=1)

In [432]:
sum(cyber_100['Employment per 1,000 jobs_impute'].isnull())

0

In [433]:
cyber_100.head()

,Area Name (short),Area Name (long),Population,Log_Population,FIPS,Annual median wage (2),"Employment per 1,000 jobs",pop_bin,Annual median wage (2)_impute,"Employment per 1,000 jobs_impute"
0,"Akron, OH","Akron, OH",698398,13.456544,10420,101130.0,0.807,7,101130.0,0.807
1,"Albany, NY","Albany-Schenectady-Troy, NY",904682,13.715339,10580,103590.0,0.978,8,103590.0,0.978
2,"Albuquerque, NM","Albuquerque, NM",922296,13.734621,10740,136290.0,2.425,8,136290.0,2.425
3,"Allentown, PA-NJ","Allentown-Bethlehem-Easton, PA-NJ",873555,13.680326,10900,119580.0,0.335,8,119580.0,0.335
4,"Atlanta, GA","Atlanta-Sandy Springs-Roswell, GA",6307261,15.657212,12060,126880.0,1.720,15,126880.0,1.720


In [434]:
#Combine with z scores and averaging

from sklearn.preprocessing import StandardScaler
import pandas as pd

# Example: your dataframe
# df = pd.read_csv("your_data.csv")  # columns: 'MSA', 'Avg_Wage', 'Emp_per_1000'

scaler = StandardScaler()
cyber_100[['wage_z', 'emp_z']] = scaler.fit_transform(cyber_100[['Annual median wage (2)_impute', 'Employment per 1,000 jobs_impute']])

# Final score: average of the two z-scores
cyber_100['Cyber_Score'] = cyber_100[['wage_z', 'emp_z']].mean(axis=1)


In [435]:
cyber_100

,Area Name (short),Area Name (long),Population,Log_Population,FIPS,Annual median wage (2),"Employment per 1,000 jobs",pop_bin,Annual median wage (2)_impute,"Employment per 1,000 jobs_impute",wage_z,emp_z,Cyber_Score
0,"Akron, OH","Akron, OH",698398,13.456544,10420,101130.0,0.807,7,101130.0,0.807,-0.617235,-0.358276,-0.487756
1,"Albany, NY","Albany-Schenectady-Troy, NY",904682,13.715339,10580,103590.0,0.978,8,103590.0,0.978,-0.478834,-0.111690,-0.295262
2,"Albuquerque, NM","Albuquerque, NM",922296,13.734621,10740,136290.0,2.425,8,136290.0,2.425,1.360893,1.974919,1.667906
3,"Allentown, PA-NJ","Allentown-Bethlehem-Easton, PA-NJ",873555,13.680326,10900,119580.0,0.335,8,119580.0,0.335,0.420776,-1.038911,-0.309068
4,"Atlanta, GA","Atlanta-Sandy Springs-Roswell, GA",6307261,15.657212,12060,126880.0,1.720,15,126880.0,1.720,0.831479,0.958292,0.894886
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Washington, DC-VA-MD","Washington-Arlington-Alexandria, DC-VA-MD-WV",6304975,15.656850,47900,138410.0,5.089,15,138410.0,5.089,1.480166,5.816471,3.648319
96,"Wichita, KS","Wichita, KS",652939,13.389239,48620,92990.0,0.834,7,92990.0,0.834,-1.075198,-0.319341,-0.697270
97,"Winston, NC","Winston-Salem, NC",695630,13.452573,49180,123700.0,1.328,7,123700.0,1.328,0.652570,0.393019,0.522794
98,"Worcester, MA-CT","Worcester, MA-CT",866866,13.672640,49340,114530.0,0.628,8,114530.0,0.628,0.136659,-0.616398,-0.239870


In [436]:
cyber_100['Cyber_Score_norm'] = (
    (cyber_100['Cyber_Score'] - cyber_100['Cyber_Score'].min()) /
    (cyber_100['Cyber_Score'].max() - cyber_100['Cyber_Score'].min())
) * 10

In [437]:
cyber_100

,Area Name (short),Area Name (long),Population,Log_Population,FIPS,Annual median wage (2),"Employment per 1,000 jobs",pop_bin,Annual median wage (2)_impute,"Employment per 1,000 jobs_impute",wage_z,emp_z,Cyber_Score,Cyber_Score_norm
0,"Akron, OH","Akron, OH",698398,13.456544,10420,101130.0,0.807,7,101130.0,0.807,-0.617235,-0.358276,-0.487756,2.380776
1,"Albany, NY","Albany-Schenectady-Troy, NY",904682,13.715339,10580,103590.0,0.978,8,103590.0,0.978,-0.478834,-0.111690,-0.295262,2.735376
2,"Albuquerque, NM","Albuquerque, NM",922296,13.734621,10740,136290.0,2.425,8,136290.0,2.425,1.360893,1.974919,1.667906,6.351805
3,"Allentown, PA-NJ","Allentown-Bethlehem-Easton, PA-NJ",873555,13.680326,10900,119580.0,0.335,8,119580.0,0.335,0.420776,-1.038911,-0.309068,2.709944
4,"Atlanta, GA","Atlanta-Sandy Springs-Roswell, GA",6307261,15.657212,12060,126880.0,1.720,15,126880.0,1.720,0.831479,0.958292,0.894886,4.927794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Washington, DC-VA-MD","Washington-Arlington-Alexandria, DC-VA-MD-WV",6304975,15.656850,47900,138410.0,5.089,15,138410.0,5.089,1.480166,5.816471,3.648319,10.000000
96,"Wichita, KS","Wichita, KS",652939,13.389239,48620,92990.0,0.834,7,92990.0,0.834,-1.075198,-0.319341,-0.697270,1.994822
97,"Winston, NC","Winston-Salem, NC",695630,13.452573,49180,123700.0,1.328,7,123700.0,1.328,0.652570,0.393019,0.522794,4.242350
98,"Worcester, MA-CT","Worcester, MA-CT",866866,13.672640,49340,114530.0,0.628,8,114530.0,0.628,0.136659,-0.616398,-0.239870,2.837416


In [438]:
cyber_100 = cyber_100[['Area Name (long)','Annual median wage (2)_impute','Employment per 1,000 jobs_impute','Cyber_Score_norm']]

In [439]:
cyber_100

,Area Name (long),Annual median wage (2)_impute,"Employment per 1,000 jobs_impute",Cyber_Score_norm
0,"Akron, OH",101130.0,0.807,2.380776
1,"Albany-Schenectady-Troy, NY",103590.0,0.978,2.735376
2,"Albuquerque, NM",136290.0,2.425,6.351805
3,"Allentown-Bethlehem-Easton, PA-NJ",119580.0,0.335,2.709944
4,"Atlanta-Sandy Springs-Roswell, GA",126880.0,1.720,4.927794
...,...,...,...,...
95,"Washington-Arlington-Alexandria, DC-VA-MD-WV",138410.0,5.089,10.000000
96,"Wichita, KS",92990.0,0.834,1.994822
97,"Winston-Salem, NC",123700.0,1.328,4.242350
98,"Worcester, MA-CT",114530.0,0.628,2.837416


In [440]:
genz = genz.merge(cyber_100,on='Area Name (long)',how='left')

In [441]:
genz

,FIPS,Area Name (short),Area Name (long),Area Type,Tarriff Impact,Industry Diversity,City economic growth,AI growth cities,F 500 cities,Job listings growth,...,Rent,Population,URate,AI Exposure,AI Exposure_norm,Entry Level % Above,Entry Level % Above_norm,Annual median wage (2)_impute,"Employment per 1,000 jobs_impute",Cyber_Score_norm
0,10420,"Akron, OH","Akron, OH",Top 100 metro areas,6.969572,7.156593,2.266667,3.333245,4.179065,1.690213,...,1274.261574,698398,5.5,0.333519,5.204906,0.630,6.267123,101130.0,0.807,2.380776
1,10580,"Albany, NY","Albany-Schenectady-Troy, NY",Top 100 metro areas,8.091453,5.787546,4.133333,5.203488,0.000000,2.744517,...,1647.776945,904682,3.5,0.348274,6.834764,0.643,6.489726,103590.0,0.978,2.735376
2,10740,"Albuquerque, NM","Albuquerque, NM",Top 100 metro areas,8.048704,6.192766,4.000000,6.085123,0.000000,3.497676,...,1633.450774,922296,3.5,0.329887,4.803682,0.652,6.643836,136290.0,2.425,6.351805
3,10900,"Allentown, PA-NJ","Allentown-Bethlehem-Easton, PA-NJ",Top 100 metro areas,6.864109,5.254121,5.733333,3.822448,2.227412,4.504602,...,1826.300472,873555,4.2,0.312858,2.922637,0.623,6.147260,119580.0,0.335,2.709944
4,12060,"Atlanta, GA","Atlanta-Sandy Springs-Roswell, GA",Top 100 metro areas,8.239089,8.983516,3.600000,7.867483,2.467971,6.184179,...,1918.618921,6307261,3.6,0.324270,4.183185,0.712,7.671233,126880.0,1.720,4.927794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,47900,"Washington, DC-VA-MD","Washington-Arlington-Alexandria, DC-VA-MD-WV",Top 100 metro areas,9.510323,4.310897,4.400000,10.000000,2.931778,3.141133,...,2429.474580,6304975,3.5,0.354736,7.548583,0.264,0.000000,138410.0,5.089,10.000000
96,48620,"Wichita, KS","Wichita, KS",Top 100 metro areas,7.852005,5.521978,4.533333,5.194047,0.000000,3.225876,...,1100.123409,652939,4.0,0.313614,3.006086,0.573,5.291096,92990.0,0.834,1.994822
97,49180,"Winston, NC","Winston-Salem, NC",Top 100 metro areas,5.677248,5.054945,2.533333,3.712683,0.000000,4.157608,...,1669.650079,695630,3.7,0.345613,6.540823,0.732,8.013699,123700.0,1.328,4.242350
98,49340,"Worcester, MA-CT","Worcester, MA-CT",Top 100 metro areas,6.595042,5.249542,1.466667,4.428149,2.244600,2.950240,...,2174.496129,866866,5.1,0.321733,3.902969,0.471,3.544521,114530.0,0.628,2.837416


In [442]:
genz.to_csv("genz_save3.csv", index=False)